In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit.primitives import Estimator
from qiskit.circuit import ParameterVector
import numpy as np
from scipy.optimize import minimize

# Generate the Hamiltonian for a specified g and number of qubits
def hamiltonian_sparse_pauli(g, num_qubits):
    pauli_terms = []
    coefficients = []

    # Add ZZ interaction terms for nearest neighbors with periodic boundary conditions
    #for i in range(num_qubits):  # Include the last pair (last qubit and first qubit)
        #label = ["I"] * num_qubits
        #label[i] = "Z"
        #label[(i + 1) % num_qubits] = "Z"  # Use modulo to wrap around for periodic boundary
        #pauli_terms.append(Pauli("".join(label)))
        #coefficients.append(-1)
    # Add ZZ interaction terms for nearest neighbors
    for i in range(num_qubits - 1):  # Exclude the last pair if no boundary conditions
        label = ["I"] * num_qubits
        label[i] = "Z"
        label[i + 1] = "Z"
        pauli_terms.append(Pauli("".join(label)))
        coefficients.append(-1)

    # Add transverse field X terms with coefficient -g
    for i in range(num_qubits):
        label = ["I"] * num_qubits
        label[i] = "X"
        pauli_terms.append(Pauli("".join(label)))
        coefficients.append(-g)

    # Create the Hamiltonian as a SparsePauliOp
    H = SparsePauliOp(pauli_terms, coefficients)
    return H

# Define the ansatz circuit
def ansatz(num_qubits):
    qc = QuantumCircuit(num_qubits)
    theta = ParameterVector('θ', num_qubits)

    # Parameterized Y rotations
    for i in range(num_qubits):
        qc.ry(theta[i], i)

    # CNOT chain entanglement
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    
    qc.cx(num_qubits - 1, 0)  # Wrap-around entanglement

    return qc, theta

# Define the cost function for optimization
def cost_function(params, hamiltonian, num_qubits):
    qc, theta = ansatz(num_qubits)
    param_dict = {theta[i]: params[i] for i in range(num_qubits)}
    bound_circuit = qc.assign_parameters(param_dict)
    
    estimator = Estimator()
    result = estimator.run(bound_circuit, [hamiltonian])
    expectation_value = result.result().values[0]
    
    return expectation_value

# Optimization loop to minimize the expectation value
def optimization_loop(hamiltonian, num_qubits, optimizer, initial_params):
    def cost(params):
        return cost_function(params, hamiltonian, num_qubits)
    
    result = minimize(cost, initial_params, method=optimizer, options={'maxiter': 1000, 'tol': 1e-6})
    return result.fun, result.x

# Set parameters for the Hamiltonian and ansatz
g = 0.8
num_qubits = 4
qubit_hamiltonian = hamiltonian_sparse_pauli(g, num_qubits)
initial_params = np.random.uniform(0, 2 * np.pi, num_qubits)
optimizer = 'COBYLA'

# Run the optimization to find the minimum energy
optimal_value, optimal_params = optimization_loop(qubit_hamiltonian, num_qubits, optimizer, initial_params)
print(f'Optimal energy: {optimal_value} Hartree')
